<a href="https://colab.research.google.com/github/mamud1977/bootup/blob/main/tensorflow_data_validation(tfdv).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow_data_validation


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 683.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-serving-api to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 69.8 MB/s 

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

In [3]:
dataset = pd.read_csv("9.0 pollution_small.csv")

In [16]:
dataset.head(4)

,Date,pm10,no2,so2,soot
0,1/1/2009,98.67,14.1,44.38,34.81
1,1/2/2009,52.33,14.1,29.75,33.06
2,1/3/2009,74.67,20.5,36.25,39.25
3,1/4/2009,72.00,17.3,46.44,34.38


In [4]:
dataset.shape

(2188, 5)

In [5]:
training_data = dataset[:1600]
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [6]:
test_data = dataset[1600:]
test_data.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Data analyis and validation with TFDV

In [12]:
training_data_stats = tfdv.generate_statistics_from_dataframe(dataframe=training_data)
#training_data_stats

In [16]:
training_data_schema = tfdv.infer_schema(statistics=training_data_stats)
#training_data_sschema

In [18]:
tfdv.display_schema(schema=training_data_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


## Compare Test data statistics

In [20]:
test_data_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_data)
test_data_schema = tfdv.infer_schema(statistics=test_data_stats)
tfdv.display_schema(schema=test_data_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


In [23]:
anomalies = tfdv.validate_statistics(statistics=test_data_stats, schema=training_data_schema)
tfdv.display_anomalies(anomalies)

In [24]:
test_data2 = test_data.copy()
test_data2.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


In [25]:
test_data2.drop("soot", axis=1,inplace=True)

In [26]:
test_data2.describe()

,pm10,no2,so2
count,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170
std,28.992087,10.940050,5.098944
min,11.900000,15.070000,4.990000
25%,28.337500,29.217500,10.122500
50%,35.555000,35.815000,12.345000
75%,50.812500,43.872500,15.855000
max,273.770000,106.030000,38.030000


In [27]:
test_data_stats2 = tfdv.generate_statistics_from_dataframe(dataframe=test_data2)
anomalies2 = tfdv.validate_statistics(statistics=test_data_stats2, schema=training_data_schema)
tfdv.display_anomalies(anomalies2)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing
